# This is the code for Quantum Optimization

In [1]:
import qiskit
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.translators import from_docplex_mp
from docplex.mp.model import Model
from qiskit_algorithms import QAOA
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit_algorithms.optimizers import COBYLA
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as SamplerV2
from qiskit.primitives import Sampler, Estimator
from qiskit_optimization.converters import QuadraticProgramToQubo
import qiskit_aer as Aer
from qiskit_aer import AerSimulator
from qiskit import transpile
from qiskit.primitives import StatevectorSampler
from qiskit_optimization.converters import InequalityToEquality

In [2]:
pip show qiskit_aer

Name: qiskit-aer
Version: 0.16.0
Summary: Aer - High performance simulators for Qiskit
Home-page: https://github.com/Qiskit/qiskit-aer
Author: AER Development Team
Author-email: qiskit@us.ibm.com
License: Apache 2.0
Location: /home/maop7/anaconda3/envs/QiskitEnv/lib/python3.9/site-packages
Requires: numpy, psutil, qiskit, scipy
Required-by: 
Note: you may need to restart the kernel to use updated packages.


## We first create the Docplex/Cplex optimization problem

In [3]:
opt_model = Model(name="MIP Model")

Nc= 3 # Nc is the number of seconds

Dist=1 # Distance to travel

tolerance = 1 # Tolerance in distance travelled

delta_v=1 # Rate of acceleration/deceleration set to 1

vmax=1 # Max speed of a TGV in France (in m/s)

alpha=0.05 # Regenerative braking efficiency

"""
We define two binary variables for two bits. 
When x=0 and y=0 then constant velocity
When x=1 and y=0 then acceleration
When x=0 and y=1 then breaking
"""

x={}
for i in range(0, Nc):
    x[i]= opt_model.binary_var(name=f"x_{i}")

y={}
for i in range(0, Nc):
    y[i]= opt_model.binary_var(name=f"y_{i}")

z={}
for i in range(0, Nc):
    z[i]= opt_model.binary_var(name=f"z_{i}")
    

objective = opt_model.linear_expr()
## objective is the hamiltonian/energy value we want to minimize
## Energy:
for i in range(0, Nc):
    objective += (delta_v**2) * x[i] - alpha*(delta_v**2)*y[i]
    #objective += (delta_v**2)*x[i]

    
opt_model.minimize(objective)

In [4]:
## Constraint 1: (simultaneous braking/acceleration)


for i in range(0, Nc):
    opt_model.add_constraint(z[i] <= x[i] , f"z_u_d_{i}") 

for i in range(0, Nc):
    opt_model.add_constraint(z[i] <= y[i], f"z_p_d_{i}")
        
for i in range(0, Nc):
    opt_model.add_constraint(z[i] >= x[i]+ y[i] -1 , f"z_u_p_d_{i}") 
    
opt_model.add_constraint(opt_model.sum(z[i] for i in range(0, Nc)) == 0 , "No_simultaneous_braking_or_acceleration_constraint")

#### Print the optimization model info

opt_model.print_information()
opt_model.prettyprint()

Model: MIP Model
 - number of variables: 9
   - binary=9, integer=0, continuous=0
 - number of constraints: 10
   - linear=10
 - parameters: defaults
 - objective: minimize
// This file has been generated by DOcplex
// model name is: MIP Model
// single vars section
dvar bool x_0;
dvar bool x_1;
dvar bool x_2;
dvar bool y_0;
dvar bool y_1;
dvar bool y_2;
dvar bool z_0;
dvar bool z_1;
dvar bool z_2;

minimize
 x_0 + x_1 + x_2 - 0.050000 y_0 - 0.050000 y_1 - 0.050000 y_2;
 
subject to {
 z_u_d_0:
  z_0 <= x_0;
 z_u_d_1:
  z_1 <= x_1;
 z_u_d_2:
  z_2 <= x_2;
 z_p_d_0:
  z_0 <= y_0;
 z_p_d_1:
  z_1 <= y_1;
 z_p_d_2:
  z_2 <= y_2;
 z_u_p_d_0:
  z_0 >= x_0 + y_0 -1;
 z_u_p_d_1:
  z_1 >= x_1 + y_1 -1;
 z_u_p_d_2:
  z_2 >= x_2 + y_2 -1;
 No_simultaneous_braking_or_acceleration_constraint:
  z_0 + z_1 + z_2 == 0;

}


In [5]:
"""
Constraint 2: (Total Distance constraints)
"""

distance = opt_model.linear_expr()
velocity = 0
for i in range(0, Nc):
    velocity = velocity + delta_v*(x[i]-y[i])
    distance += velocity
'''opt_model.add_constraint(distance <= Dist+tolerance, "Max_Distance_constraint")
opt_model.add_constraint(distance >= Dist-tolerance, "Min_Distance_constraint")'''
opt_model.add_constraint(distance == Dist, "Min_Distance_constraint")

#### Print the optimization model info

opt_model.print_information() 

Model: MIP Model
 - number of variables: 9
   - binary=9, integer=0, continuous=0
 - number of constraints: 11
   - linear=11
 - parameters: defaults
 - objective: minimize


In [6]:
"""
Constraint 3: (Net-Zero contraint)

"""
opt_model.add_constraint(opt_model.sum((y[i]-x[i]) for i in range(0, Nc)) == 0 , "Net_Zero_constraint")

#### Print the optimization model info

opt_model.print_information()

Model: MIP Model
 - number of variables: 9
   - binary=9, integer=0, continuous=0
 - number of constraints: 12
   - linear=12
 - parameters: defaults
 - objective: minimize


In [7]:
"""
Constraint 4: (Maximum Speed)

"""
opt_model.add_constraint(opt_model.sum((delta_v*x[i]) for i in range(0, Nc)) <= vmax , "Maximum_Speed_constraint")

#### Print the optimization model info

opt_model.print_information() 

Model: MIP Model
 - number of variables: 9
   - binary=9, integer=0, continuous=0
 - number of constraints: 13
   - linear=13
 - parameters: defaults
 - objective: minimize


In [8]:
'''
#Constraint 5: (Positive Speed)
for i in range(Nc):
    opt_model.add_constraint(opt_model.sum((x[i]-y[i]) for i in range(0, i)) >= 0 , "Positive_Speed_constraint")

    #### Print the optimization model

opt_model.print_information()
'''

'\n#Constraint 5: (Positive Speed)\nfor i in range(Nc):\n    opt_model.add_constraint(opt_model.sum((x[i]-y[i]) for i in range(0, i)) >= 0 , "Positive_Speed_constraint")\n\n    #### Print the optimization model\n\nopt_model.print_information()\n'

In [9]:
'''
#Constraint 6: (Must leave immediately)

opt_model.add_constraint(x[0]==1, "Must leave immediately constraint")
    

#### Print the optimization model info

opt_model.print_information()
'''

'\n#Constraint 6: (Must leave immediately)\n\nopt_model.add_constraint(x[0]==1, "Must leave immediately constraint")\n    \n\n#### Print the optimization model info\n\nopt_model.print_information()\n'

In [10]:
#### Print the optimization model
print(opt_model.prettyprint())

// This file has been generated by DOcplex
// model name is: MIP Model
// single vars section
dvar bool x_0;
dvar bool x_1;
dvar bool x_2;
dvar bool y_0;
dvar bool y_1;
dvar bool y_2;
dvar bool z_0;
dvar bool z_1;
dvar bool z_2;

minimize
 x_0 + x_1 + x_2 - 0.050000 y_0 - 0.050000 y_1 - 0.050000 y_2;
 
subject to {
 z_u_d_0:
  z_0 <= x_0;
 z_u_d_1:
  z_1 <= x_1;
 z_u_d_2:
  z_2 <= x_2;
 z_p_d_0:
  z_0 <= y_0;
 z_p_d_1:
  z_1 <= y_1;
 z_p_d_2:
  z_2 <= y_2;
 z_u_p_d_0:
  z_0 >= x_0 + y_0 -1;
 z_u_p_d_1:
  z_1 >= x_1 + y_1 -1;
 z_u_p_d_2:
  z_2 >= x_2 + y_2 -1;
 No_simultaneous_braking_or_acceleration_constraint:
  z_0 + z_1 + z_2 == 0;
 Min_Distance_constraint:
  3 x_0 - 3 y_0 + 2 x_1 - 2 y_1 + x_2 - y_2 == 1;
 Net_Zero_constraint:
  y_0 - x_0 + y_1 - x_1 + y_2 - x_2 == 0;
 Maximum_Speed_constraint:
  x_0 + x_1 + x_2 <= 1;

}
None


## Problem conversion

In [11]:
# Conversion to qad_model
qp_quad = from_docplex_mp(opt_model)
print("Number of variables:", len(qp_quad.variables))

Number of variables: 9


In [12]:
ineq2eq = InequalityToEquality()
qp_eq = ineq2eq.convert(qp_quad)


# Conversion to qubo
conv = QuadraticProgramToQubo()
qubo = conv.convert(qp_eq)

print(f"QUBO variables: {len(qubo.variables)}")
print(f"QUBO constraints: {len(qubo.linear_constraints)}")

QUBO variables: 22
QUBO constraints: 0


In [13]:
qubitOp, offset = qubo.to_ising()
print("Offset:", offset)
print("Ising Hamiltonian:")
print(str(qubitOp))



Offset: 105.17499999999988
Ising Hamiltonian:
SparsePauliOp(['IIIIIIIIIIIIIIIIIIIIIZ', 'IIIIIIIIIIIIIIIIIIZIII', 'IIIIIIIIIIIIIIIIIIIIZI', 'IIIIIIIIIIIIIIIIIZIIII', 'IIIIIIIIIIIIIIIIIIIZII', 'IIIIIIIIIIIIIIIIZIIIII', 'IIIIIIIIIIIIIIIZIIIIII', 'IIIIIIIIIIIIIIZIIIIIII', 'IIIIIIIIIIIIIZIIIIIIII', 'IIIIIIZIIIIIIIIIIIIIII', 'IIIIIZIIIIIIIIIIIIIIII', 'IIIIZIIIIIIIIIIIIIIIII', 'IIIZIIIIIIIIIIIIIIIIII', 'IIZIIIIIIIIIIIIIIIIIII', 'IZIIIIIIIIIIIIIIIIIIII', 'ZIIIIIIIIIIIIIIIIIIIII', 'IIIIIIIIIIIIIIIIIIIIZZ', 'IIIIIIIIIIIIIIIIIIIZIZ', 'IIIIIIIIIIIIIIIIIIZIIZ', 'IIIIIIIIIIIIIIIIIZIIIZ', 'IIIIIIIIIIIIIIIIZIIIIZ', 'IIIIIIIIIIIIIIIZIIIIIZ', 'IIIIIIIIIIIIZIIIIIIIIZ', 'IIIIIIIIIIIIZIIIIIIIII', 'IIIIIIZIIIIIIIIIIIIIIZ', 'IIIIIZIIIIIIIIIIIIIIIZ', 'ZIIIIIIIIIIIIIIIIIIIIZ', 'IIIIIIIIIIIIIIIIIIIZZI', 'IIIIIIIIIIIIIIIIIIZIZI', 'IIIIIIIIIIIIIIIIIZIIZI', 'IIIIIIIIIIIIIIIIZIIIZI', 'IIIIIIIIIIIIIIZIIIIIZI', 'IIIIIIIIIIIZIIIIIIIIZI', 'IIIIIIIIIIIZIIIIIIIIII', 'IIIIZIIIIIIIIIIIIIIIZI', 'IIIZIIIIIIIIIIIIIIIIZI', 'ZI

### Running on a QC simulator

In [14]:
simulator = AerSimulator(method='statevector', device='GPU')

In [15]:
# Configuring solver
qaoa = QAOA(sampler=Estimator(), optimizer=COBYLA(maxiter=10),reps=1)
# Set options for quantum instance
backend_options = {
	"method": "statevector",
	"optimization_level": 1,
}

# Transpile circuit for simulator
qaoa.quantum_instance = simulator
meo = MinimumEigenOptimizer(qaoa)

/tmp/ipykernel_7552/1719759574.py:2: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  qaoa = QAOA(sampler=Estimator(), optimizer=COBYLA(maxiter=10),reps=1)


In [16]:
# Solving
result_qaoa = meo.solve(qubo)
print(result_qaoa.x)
print(result_qaoa.fval)

QiskitError: 'Invalid input data for Pauli.'

In [33]:
'''backend = AerSimulator(method="statevector")
print(backend.available_devices())
optimizer = COBYLA(maxiter=100)
qaoa = QAOA(optimizer=optimizer, reps=1,sampler=Sampler)'''

('CPU',)


In [ ]:
'''qaoa_optimizer = MinimumEigenOptimizer(qaoa)
result = qaoa_optimizer.solve(qubo)
# Print the solution
print("Solution (bitstring):", result.x)
print("Objective value (cut size):", result.fval)'''

### Running on a Quantum Computer

In [67]:
'''from qiskit_ibm_runtime import QiskitRuntimeService
# Load saved credentials
service = QiskitRuntimeService()
# Get the least busy backend
backend = service.least_busy(operational=True, simulator=False)
print("least busy backend: ", backend)'''

least busy backend:  <IBMBackend('ibm_kyiv')>


In [6]:
'''sampler=Sampler(mode=backend)
optimizer=COBYLA(maxiter=100)
qaoa=QAOA(sampler=sampler, optimizer=optimizer)
meo = MinimumEigenOptimizer(qaoa)
result = meo.solve(qubitOp)
print(result.prettyprint())'''

'sampler=Sampler(mode=backend)\noptimizer=COBYLA(maxiter=100)\nqaoa=QAOA(sampler=sampler, optimizer=optimizer)\nmeo = MinimumEigenOptimizer(qaoa)\nresult = meo.solve(qubitOp)\nprint(result.prettyprint())'